# Преобразование персональных данных

Нужно защитить данные клиентов страховой компании, преобразовав их таким образом, чтобы по ним было сложно восстановить персональную информацию. При этом качество моделей машинного обучения не должно ухудшиться.

## 1. Загрузка данных

In [1]:
#импортируем библиотеки
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
#считываем датасет
#Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
#Целевой признак: количество страховых выплат клиенту за последние 5 лет.

data_ins = pd.read_csv('/datasets/insurance.csv')

In [3]:
data_ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data_ins.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
data_ins.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
data_ins.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [7]:
#Разбиваем датасет на признаки и целевой признак
features = data_ins.drop('Страховые выплаты', axis=1)
target = data_ins['Страховые выплаты']

## 2. Умножение матриц
Отвечаем на вопрос, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу.

In [8]:
#Признаки переводим в матрицу
feat_matrix = np.array(features)

#Целевые признаки переводим в вектор
targ_vector = np.array(target)
#print(targ_vector)

In [9]:
#Замеряем MSE и R2
print('Качество линейной регрессии')

model = LinearRegression()
model.fit(feat_matrix, targ_vector)
pred_vector = model.predict(feat_matrix)
mse = mean_squared_error(targ_vector,pred_vector)
print('MSE=',mse)              
rmse = mse ** 0.5
print('RMSE=',rmse)            
r2 = r2_score(targ_vector, pred_vector)
print("R2 =", r2)

Качество линейной регрессии
MSE= 0.1233468894171086
RMSE= 0.3512077581960692
R2 = 0.42494550286668


In [10]:
##Генерим случайную матрицу и проверяем на обратимость
rand_matrix_size = feat_matrix.shape[1]
good_res = False
while good_res != True:
    rand_matrix = np.random.normal(0,1,(rand_matrix_size,rand_matrix_size))
    try:
        rand_matrix_invert = np.linalg.inv(rand_matrix)
        good_res = True   
    except np.linalg.LinAlgError:
        continue

In [11]:
#Умножаем признаки на обратную матрицу
feat_matrix_times_rand_matrix = feat_matrix @ rand_matrix
#print(feat_matrix_times_rand_matrix)

In [12]:
#Замеряем MSE и R2 после умножения признаков на обратную матрицу
print('Качество линейной регрессии после умножения признаков на обратимую матрицу')
model = LinearRegression()
model.fit(feat_matrix_times_rand_matrix, targ_vector)
pred_vector = model.predict(feat_matrix_times_rand_matrix)
mse = mean_squared_error(targ_vector,pred_vector)
print('MSE_2=',mse)              
rmse = mse ** 0.5
print('RMSE_2=',rmse)            
r2 = r2_score(targ_vector, pred_vector)
print("R2_2 =", r2)

Качество линейной регрессии после умножения признаков на обратимую матрицу
MSE_2= 0.12334688941710797
RMSE_2= 0.3512077581960683
R2_2 = 0.424945502866683


In [13]:
#теперь пошаговый расчет на небольшой матрице
#для начальной матрицы возьмем первые пять записей признаков исходного датасета
matrix = np.array([[1,41.0,49600.0,1],[0,46.0,38000.0,1],[0,29.0,21000.0,0],[0,21.0,41700.0,2],[1,28.0,26100.0,0]])
print('начальная матрица\n',matrix)
print()

#создадим случайным образом обратимую матрицу 
rand_matrix = np.random.normal(1,0.1,(4,4))
print('обратимая матрица, на которую умножим начальную\n',rand_matrix)
print()
#проверим полученую матрицу на обратимость, если ошибок не будет, значит обратимая
rand_matrix_inv = np.linalg.inv(rand_matrix)

#перемножим первую и обратимую матрицы
crypto = matrix @ rand_matrix
print('перемножили начальную и обратимую, полученную матрицу назовем крипто-матрицей для удобства\n',crypto)
print()

#создадим целевой вектор
y = np.array([0,1,0,0,0])

#w - вектор весов
#w0 - величина сдвига
#график предсказаний: y = w*X + w0
#задача обучения - найти w и w0

# добавляем столбец с 1 в признаки
X = np.concatenate((np.ones((5, 1)), matrix), axis=1)
print('Добавляем к признакам начальной матрицы столбец с 1 \n',X)

crypto_X = np.concatenate((np.ones((5, 1)),crypto), axis=1)
print('Добавляем к признакам крипто-матрицы столбец с 1 \n',crypto_X)
print()

print('Целевой признак y\n',y)
print()

#MSE минимальна, когда веса рассчитаем по формуле w = np.linalg.inv(X.T @ X) @ X.T @ y
x_t = X.T
print('Транспонируем начальную матрицу (X)\n',x_t)
crypto_x_t = crypto_X.T
print('Транспонируем крипто-матрицу (crypto_X)\n',crypto_x_t)
print()

x_t_x = x_t @ X
print('Перемножаем X.T и X \n',x_t_x)
crypto_x_t_x = crypto_x_t @ crypto_X
print('Перемножаем crypto_X.T и crypto_X \n',crypto_x_t_x)
print()

x_t_x_inv = np.linalg.inv(x_t_x)
print('Получаем обратную матрицу от перемноженной X.T и X\n',x_t_x_inv)
crypto_x_t_x_inv = np.linalg.inv(crypto_x_t_x)
print('Получаем обратную крипто-матрицу от перемноженной crypto_X.T и crypto_X\n',crypto_x_t_x_inv)
print()

x_t_inv_x_t = x_t_x_inv @ x_t
print('Перемножаем обратную и транспонированную X\n',x_t_inv_x_t)
crypto_x_t_inv_x_t = crypto_x_t_x_inv @ crypto_x_t
print('Перемножаем обратную крипто-матрицу и транспонированную crypto_X\n',crypto_x_t_inv_x_t)
print()

wesa = x_t_inv_x_t @ y
print('Считаем веса\n',wesa)
crypto_wesa = crypto_x_t_inv_x_t @ y
print('Считаем крипто-веса\n',crypto_wesa)
print()
#print('Проверим по формуле w = np.linalg.inv(X.T @ X) @ X.T @ y \n',np.linalg.inv(X.T @ X) @ X.T @ y)
#print('Проверим крипто по формуле w = np.linalg.inv(X.T @ X) @ X.T @ y \n',np.linalg.inv(crypto_X.T @ crypto_X) @ crypto_X.T @ y)

w = wesa[1:]
print('w (веса)\n',w)
crypto_w = crypto_wesa[1:]
print('crypto_w (крипто-веса)\n',crypto_w)
print()

w0 = wesa[0]
print('w0 (сдвиг)\n',w0)
crypto_w0 = crypto_wesa[0]
print('crypto_w0 (крипто-сдвиг)\n',crypto_w0)
print()

#генерим предсказания
pred = matrix @ w + w0   
print('Предсказания\n',pred)
crypto_pred = crypto @ crypto_w + crypto_w0   
print('Крипто-предсказания\n',crypto_pred)
print()
       
r2 = r2_score(y, pred)
print("R2 =", r2)
crypto_r2 = r2_score(y, crypto_pred)
print("Крипто-R2 =", crypto_r2)

начальная матрица
 [[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 [0.00e+00 2.10e+01 4.17e+04 2.00e+00]
 [1.00e+00 2.80e+01 2.61e+04 0.00e+00]]

обратимая матрица, на которую умножим начальную
 [[1.14644376 0.88772754 1.02124626 0.97780466]
 [0.89686727 0.85761913 0.86337848 0.96138749]
 [1.01994672 1.07245624 0.85409876 0.99997263]
 [0.8633178  0.88104866 0.8791     1.05349929]]

перемножили начальную и обратимую, полученную матрицу назовем крипто-матрицей для удобства
 [[50628.13878011 53230.76047499 42400.59747601 49640.09050911]
 [38800.094686   40793.66850345 32496.34747908 38044.23716435]
 [21444.89033372 22546.45191454 17961.11198512 21027.30541225]
 [42552.33919707 44741.19714766 35635.8075376  41721.15469758]
 [26646.86819746 28016.00882746 22317.17354081 26127.18222903]]

Добавляем к признакам начальной матрицы столбец с 1 
 [[1.00e+00 1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [1.00e+00 0.00e+00 4.60e+01 3.80e

**Ответ:**

Качество линейной регрессии не изменится.

**Обоснование:** 

Проведя обучение линейной регрессии на исходном датасете, а затем на исходном датасете с измененными признаками (путем умножения их на обратимую матрицу) и сравнив полученные качественные характеристики, мы выяснили, что качество линейной регрессии после преобразования признаков не меняется. 
Чтобы выяснить, почему это происходит, провели эксперимент с небольшой матрицей, пройдя по всем расчетам линейной регрессии.
Насколько можно судить, качество регрессии не меняется, поскольку изменение признаков компенсируется изменением расчетных значений весов (w и w0), при которых MSE будет минимальна.

## 3. Алгоритм преобразования
Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

1. Преобразуем признаки в матрицу, а таргет в вектор.
2. Сгенерируем случайным образом обратимую матрицу.
3. Получим новую матрицу путем умножения матрицы признаков на рандомную обратимую матрицу.
3. Переведем новую матрицу в датасет.
4. PROFIT: данные преобразованы, персональную информацию по ним уже не восстановить, качество линейной регрессии при этом не пострадало, оставшись на прежнем уровне.

**Обоснование**

Качество линейной регрессии не меняется, потому что изменение признаков влечет за собой соответствующее изменение рассчитываемых вектора весов w и сдвига предсказания w0, при которых MSE остается минимальной.

## 4. Проверка алгоритма
Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [14]:
#Функция преобразования признаков
#генерим рандомную обратимую матрицу, умножаем признаки на нее, возвращаем результат
def kripto(features):
    rand_matrix_size = features.shape[1]
    good_res = False
    while good_res != True:
        rand_matrix = np.random.normal(0,1,(rand_matrix_size,rand_matrix_size))
        try:
            rand_matrix_invert = np.linalg.inv(rand_matrix)
            good_res = True   
        except np.linalg.LinAlgError:
            continue        
    kripto_matrix = features @ rand_matrix
    return kripto_matrix

In [15]:
#функция замера результатов
def our_score(features,target):
    
    model = LinearRegression()
    model.fit(features, target)
    predicted = model.predict(features)

    mse = mean_squared_error(target,predicted)
    print('MSE= {:.6f}'.format(mse))              
    rmse = mse ** 0.5
    print('RMSE= {:.6f}'.format(rmse))            
    r2 = r2_score(target, predicted)
    print("R2= {:.6f}".format(r2))    

In [16]:
#Выполняем алгоритм

#Признаки переводим в матрицу
feat_matrix = np.array(features)
#Целевые признаки переводим в вектор
targ_vector = np.array(target)

#получаем результаты для признаков
print('Качество линейной регрессии')
our_score(feat_matrix,targ_vector)

#преобразуем признаки
kripto_feat = kripto(feat_matrix)

#получаем результаты для преобразованых признаков
print('\nКачество линейной регрессии после преобразования признаков')
our_score(kripto_feat,targ_vector)

#преобразуем матрицу преобразованных признаков в датасет
data_kripto_features = pd.DataFrame(kripto_feat, columns=list(features))
data_kripto_features.head()

Качество линейной регрессии
MSE= 0.123347
RMSE= 0.351208
R2= 0.424946

Качество линейной регрессии после преобразования признаков
MSE= 0.123347
RMSE= 0.351208
R2= 0.424946


,Пол,Возраст,Зарплата,Члены семьи
0,50249.737719,58223.348097,-51594.509394,-71558.772152
1,38507.742556,44603.957745,-39533.610215,-54832.077592
2,21282.225329,24649.407782,-21847.968210,-30304.289746
3,42239.175008,48950.117503,-43374.672664,-60151.857195
4,26445.288795,30637.626488,-27150.537143,-37659.481044


**Выводы**
Алгоритм преобразования данных реализован, в результате его работы данные преобразуются так, что персональную информацию по ним уже не восстановить, качество линейной регрессии при этом остается на прежнем уровне.